In [1]:
from pyscal_rdf.network.network import OntologyNetwork

In [2]:
prov = OntologyNetwork('../pyscal_rdf/data/prov.rdf', delimiter='#')
cmso = OntologyNetwork('../pyscal_rdf/data/cmso.owl')
pldo = OntologyNetwork('../pyscal_rdf/data/pldo.owl')
podo = OntologyNetwork('../pyscal_rdf/data/podo.owl')
msmo = OntologyNetwork('../pyscal_rdf/data/msmo.owl')

* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#specializationOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov.alternateOf]; I'm trying to fix it...
* Owlready2 * WARNING: ObjectProperty http://www.w3.org/ns/prov#wasRevisionOf belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, prov.wasDerivedFrom]; I'm trying to fix it...


In [3]:
combo = cmso + pldo + podo + msmo

Manually add provo attributes

In [4]:
combo.add_namespace('prov', 'http://www.w3.org/ns/prov#')
combo.add_namespace('rdf', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#')
combo.add_namespace('rdfs', 'http://www.w3.org/2000/01/rdf-schema#')

In [5]:
combo.namespaces

{'cmso': 'http://purls.helmholtz-metadaten.de/cmso/',
 'qudt': 'http://qudt.org/schema/qudt/',
 'pldo': 'http://purls.helmholtz-metadaten.de/pldo/',
 'podo': 'http://purls.helmholtz-metadaten.de/podo/',
 'msmo': 'http://purls.helmholtz-metadaten.de/msmo/',
 'prov': 'http://www.w3.org/ns/prov#',
 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'}

In [6]:
combo.add_term('http://www.w3.org/ns/prov#Entity', 'class', delimiter='#')
combo.add_term('http://www.w3.org/ns/prov#Activity', 'class', delimiter='#')
combo.add_term('http://www.w3.org/ns/prov#SoftwareAgent', 'class', delimiter='#')
combo.add_term('http://www.w3.org/ns/prov#wasDerivedFrom', 'object_property', delimiter='#')
combo.add_term('http://www.w3.org/ns/prov#wasGeneratedBy', 'object_property', delimiter='#')
combo.add_term('http://www.w3.org/ns/prov#wasAssociatedWith', 'object_property', delimiter='#')
combo.add_term('http://www.w3.org/ns/prov#actedOnBehalfOf', 'object_property', delimiter='#')
combo.add_term('http://www.w3.org/2000/01/rdf-schema#label', 'data_property', delimiter='#', namespace='rdfs')
combo.add_term('http://www.w3.org/1999/02/22-rdf-syntax-ns#type', 'object_property', delimiter='#', namespace='rdf')

In [7]:
combo.add_path(('cmso:Material', 'cmso:hasDefect', 'pldo:PlanarDefect'))
combo.add_path(('cmso:Material', 'cmso:hasDefect', 'podo:Vacancy'))
combo.add_path(('cmso:SimulationCell', 'podo:hasVacancyConcentration', 'float'))
combo.add_path(('cmso:SimulationCell', 'podo:hasNumberOfVacancies', 'int'))
combo.add_path(('cmso:ComputationalSample', 'prov:wasDerivedFrom', 'cmso:ComputationalSample'))
combo.add_path(('cmso:ComputationalSample', 'prov:wasGeneratedBy', 'msmo:ComputationalMethod'))
combo.add_path(('msmo:ComputationalMethod', 'prov:wasAssociatedWith', 'prov:SoftwareAgent'))
combo.add_path(('cmso:ComputationalSample', 'rdf:type', 'prov:Entity'))
combo.add_path(('msmo:ComputationalMethod', 'rdf:type', 'prov:Activity'))


Test a complex workflow query

In [8]:
s = combo.create_query(combo.terms.cmso.ComputationalSample, [combo.terms.msmo.ComputationalMethod, combo.terms.prov.SoftwareAgent])
print(s)

PREFIX cmso: <http://purls.helmholtz-metadaten.de/cmso/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX pldo: <http://purls.helmholtz-metadaten.de/pldo/>
PREFIX podo: <http://purls.helmholtz-metadaten.de/podo/>
PREFIX msmo: <http://purls.helmholtz-metadaten.de/msmo/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?ComputationalMethod ?SoftwareAgent
WHERE {
    ?ComputationalSample prov:wasGeneratedBy ?ComputationalMethod .
    ?ComputationalSample prov:wasGeneratedBy ?AbinitioMolecularDynamics .
    ?AbinitioMolecularDynamics prov:wasAssociatedWith ?SoftwareAgent .
FILTER 
}


Test a query

In [17]:
condition = (combo.terms.pldo.hasSigmaValue > 5)

In [18]:
s = combo.create_query(combo.terms.cmso.ComputationalSample, 
                   [combo.terms.pldo.hasSigmaValue])
print(s)

PREFIX cmso: <http://purls.helmholtz-metadaten.de/cmso/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX pldo: <http://purls.helmholtz-metadaten.de/pldo/>
PREFIX podo: <http://purls.helmholtz-metadaten.de/podo/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX msmo: <http://purls.helmholtz-metadaten.de/msmo/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?hasSigmaValuevalue
WHERE {
    ?ComputationalSample cmso:hasMaterial ?AmorphousMaterial .
    ?AmorphousMaterial cmso:hasDefect ?AntiphaseBoundary .
    ?AntiphaseBoundary pldo:hasSigmaValue ?hasSigmaValuevalue .
FILTER 
}


In [ ]:
q = cmso.create_query('cmso:ComputationalSample', ['pldo:hasSigmaValuevalue', 'cmso:hasAngle_alphavalue'], values = ['val > 5 and val < 10'])
print(q)

In [28]:
from rdflib import RDFS, RDF

In [29]:
RDF.type

rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type')